https://deepmlblog.wordpress.com/2016/01/05/residual-networks-in-torch-mnist/

In [39]:
require 'torch'
require 'nn'
require 'optim'
require 'image'
torch.manualSeed(1)

function reload(name)
    package.loaded[name] = nil
    return require(name)
end
utils = reload('./utils')
models = reload('./models')
Logger = reload('./logger')


In [41]:

train_data, train_label,
    validation_data, validation_label = utils.load_data()

train_data = train_data[{{1,1000}}]
train_label = train_label[{{1, 1000}}]


In [135]:
print('train')
print(train_data:mean())
print(train_data:std())
print(train_data:min())
print(train_data:max())
print('validation')
print(validation_data:mean())
print(validation_data:std())
print(validation_data:min())
print(validation_data:max())


train	
-0.0078593129846774	


0.99036449947161	


-0.42407388981865	
2.8215433152638	
validation	


-0.0048816247909239	


0.99368015338313	


-0.42407388981865	


2.8215433152638	


In [42]:

SEED = 1
MODEL_NAME = 'mlp_3'
TITLE = 'mnist_' .. MODEL_NAME
-- VARIANT = 1
-- LOGPATH = './results/' .. TITLE .. '/' .. VARIANT
EPOCHS = 10

p1 = {
   learningRate = 1e-1,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p2 = {
   learningRate = 1e-1,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p3 = {
   learningRate = 1e-2,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p4 = {
   learningRate = 1e-2,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p5 = {
   learningRate = 1e-3,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p6 = {
   learningRate = 1e-3,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

p7 = {
   learningRate = 1e-4,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

p8 = {
   learningRate = 1e-4,
   learningRateDecay = 0,
   weightDecay = 0.2,
   momentum = 0
}

sgd_params = {p1, p2, p3, p4, p5, p6, p7, p8}

batch_params = {
    n_of_batches = 100
}

-- for i=1,1 do
for i=1,#sgd_params do
    VARIANT = i
    LOGPATH = './results/' .. TITLE .. '/' .. VARIANT
    start_training(sgd_params[i], batch_params)
end


In [30]:

function start_training(sgd_params, batch_params)
    
    torch.manualSeed(SEED)
    
    local n_of_batches = batch_params.n_of_batches
    
    model = models[MODEL_NAME]()
    criterion = nn.ClassNLLCriterion()

    x, dl_params = model:getParameters()

    info_log = Logger(
        LOGPATH, 
        {
            general_info = {
                title = TITLE,
                description = '',
                seed = SEED,
                model = tostring(model),
                criterion = tostring(criterion),
            },
            data_size = {
                train_data = torch.totable(train_data:size()),
                train_label = torch.totable(train_label:size()),
                validation_data = torch.totable(validation_data:size()),
                validation_label = torch.totable(validation_label:size()),
            },
            sgd_params = sgd_params
        })
    info_log:write()

    logger = optim.Logger(LOGPATH .. '/accuracy.log')
    logger:setNames{'train_loss', 'validation_loss'}

    time_start = os.time()
    
    batch_size = train_data:size()[1] / n_of_batches
    
    for epoch=1,EPOCHS do
        
        assert(train_data:mean() > -0.1 and train_data:mean() < 0.1)
        assert(validation_data:mean() > -0.1 and validation_data:mean() < 0.1)

        train_loss = 0
        
        for j=1,n_of_batches do
            batch_data = utils.get_batch(train_data, j, batch_size)
            batch_label = utils.get_batch(train_label, j, batch_size)

            sys.tic()
            _, fs = optim.sgd(utils.feval,x,sgd_params)
            duration = sys.toc()
            train_loss = train_loss + fs[1]
        end
        
        train_loss = train_loss/n_of_batches

        validation_loss = calc_validation_loss(model, criterion, validation_data, validation_label)
        logger:add{train_loss, validation_loss}

    end

    total_train_time = os.time() - time_start
    
    train_accuracy = utils.calc_accuracy(model, train_data, train_label)
    validation_accuracy = utils.calc_accuracy(model, validation_data, validation_label)
    
    info_log:write_result({
            model_name = MODEL_NAME,
            variant = VARIANT,
            epochs = EPOCHS,
            learningRate = sgd_params.learningRate,
            weightDecay = sgd_params.weightDecay,
            total_train_time = total_train_time,
            train_loss = train_loss,
            validation_loss = validation_loss,
            train_accuracy = train_accuracy,
            validation_accuracy = validation_accuracy,
        })
--     info_log:write_result(
--         total_train_time, train_loss, validation_loss, train_accuracy, validation_accuracy
--     )

    logger:style{'+-', '+-'}
    logger:plot()
end


In [10]:
-- tbl = {
--     a = 'ss'
-- }
-- json = require 'json'
-- -- decode(tbl)
-- print(json)


In [27]:
json.encode({abcd='a', we=2}, 2, 2)

{"abcd":"a","we":2}	
